# Customer Behaivor Prediction (2) - Monthly Spending

## Overview
In 1998, the Adventure Works Cycles company collected a large volume of data about their existing customers, including demographic features and information about purchases they have made. The company is particularly interested in analyzing customer data to determine any apparent relationships between demographic features known about the customers and the likelihood of a customer purchasing a bike. Additionally, the analysis should endeavor to determine whether a customer's average monthly spend with the company can be predicted from known customer characteristics.

## Goal
The goal is to build a regression model to predict customers' monthly spending.

## Data
The training and test datasets are retrieved from [Microsoft Learning](https://github.com/MicrosoftLearning).

## Model
We are adopting a simple linear regression model to predict customers' monthly spending. Evaluated with cross-validation on the training data, the simple model gives an R^2 of **0.9464**.

## Import the libraries

In [8]:
import pandas as pd
import numpy as np
import numpy.random as nr
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn import linear_model
import sklearn.metrics as sklm
import seaborn as sns
import matplotlib.pyplot as plt
import math
import scipy.stats as ss
from sklearn.model_selection import KFold, cross_val_score

## Load the data

In [2]:
# LOAD DATA
custs = pd.read_csv('D:\\Users\\user\\Desktop\\MPP DS Cert\\DAT275x - Principles of Machine Learning Python Edition\\AdvWorksCusts.csv')
avgms = pd.read_csv('D:\\Users\\user\\Desktop\\MPP DS Cert\\DAT275x - Principles of Machine Learning Python Edition\\AW_AveMonthSpend.csv')
bikeby = pd.read_csv('D:\\Users\\user\\Desktop\\MPP DS Cert\\DAT275x - Principles of Machine Learning Python Edition\\AW_BikeBuyer.csv')
AW_test = pd.read_csv('D:\\Users\\user\\Desktop\\MPP DS Cert\\DAT275x - Principles of Machine Learning Python Edition\\AW_test.csv')

## Drop duplicates

In [3]:
#DROP DUPLICATES
custs = custs.drop_duplicates(keep = 'last')
avgms = avgms.drop_duplicates(keep = 'last')
bikeby = bikeby.drop_duplicates(keep = 'last')

In [4]:
#MERGE CSVS
custs = pd.merge(custs, avgms, on='CustomerID')
custs = pd.merge(custs, bikeby, on='CustomerID')
# print(custs.info())

In [6]:
custs.columns

Index(['CustomerID', 'Title', 'FirstName', 'MiddleName', 'LastName', 'Suffix',
       'AddressLine1', 'AddressLine2', 'City', 'StateProvinceName',
       'CountryRegionName', 'PostalCode', 'PhoneNumber', 'BirthDate',
       'Education', 'Occupation', 'Gender', 'MaritalStatus', 'HomeOwnerFlag',
       'NumberCarsOwned', 'NumberChildrenAtHome', 'TotalChildren',
       'YearlyIncome', 'AveMonthSpend', 'BikeBuyer'],
      dtype='object')

## Feature engineering

In [9]:
#CREATE AGE
custs['Today'] = pd.Timestamp("1998-01-01")
custs['BirthDate'] = pd.to_datetime(custs['BirthDate'])
custs['Age'] = custs['Today'] - custs['BirthDate']
custs['Age'] = custs['Age']/np.timedelta64(1,'Y')
custs['Age'] = custs['Age'].round()

#CREATE AGE for test data
AW_test['Today'] = pd.Timestamp("1998-01-01")
AW_test['BirthDate'] = pd.to_datetime(AW_test['BirthDate'])
AW_test['Age'] = AW_test['Today'] - AW_test['BirthDate']
AW_test['Age'] = AW_test['Age']/np.timedelta64(1,'Y')
AW_test['Age'] = AW_test['Age'].round()

print(custs['Age'])

0        32.0
1        33.0
2        32.0
3        30.0
4        29.0
5        32.0
6        32.0
7        34.0
8        33.0
9        34.0
10       34.0
11       34.0
12       30.0
13       29.0
14       30.0
15       19.0
16       19.0
17       20.0
18       19.0
19       19.0
20       19.0
21       52.0
22       52.0
23       51.0
24       52.0
25       50.0
26       51.0
27       50.0
28       51.0
29       50.0
         ... 
16441    20.0
16442    21.0
16443    54.0
16444    36.0
16445    37.0
16446    37.0
16447    37.0
16448    38.0
16449    38.0
16450    38.0
16451    38.0
16452    38.0
16453    39.0
16454    38.0
16455    39.0
16456    39.0
16457    37.0
16458    38.0
16459    39.0
16460    41.0
16461    42.0
16462    43.0
16463    46.0
16464    50.0
16465    41.0
16466    33.0
16467    62.0
16468    58.0
16469    51.0
16470    53.0
Name: Age, Length: 16471, dtype: float64


In [10]:
#DROP COLUMNS
custs.drop('Title', axis = 1, inplace = True)
custs.drop('MiddleName', axis = 1, inplace = True)
custs.drop('Suffix', axis = 1, inplace = True)
custs.drop('AddressLine2', axis = 1, inplace = True)

print(custs.shape)
print(custs.head)

#CHECK FOR CLASS IMBALANCE
bb_counts = custs[['CustomerID', 'BikeBuyer']].groupby('BikeBuyer').count()

(16471, 23)
<bound method NDFrame.head of        CustomerID   FirstName    LastName                 AddressLine1  \
0           11000         Jon        Yang              3761 N. 14th St   
1           11001      Eugene       Huang                   2243 W St.   
2           11002       Ruben      Torres             5844 Linden Land   
3           11003     Christy         Zhu             1825 Village Pl.   
4           11004   Elizabeth     Johnson          7553 Harness Circle   
5           11005       Julio        Ruiz          7305 Humphrey Drive   
6           11006       Janet     Alvarez                2612 Berry Dr   
7           11007       Marco       Mehta             942 Brook Street   
8           11008         Rob     Verhoff             624 Peabody Road   
9           11009     Shannon     Carlson          3839 Northgate Road   
10          11010   Jacquelyn      Suarez          7800 Corrinne Court   
11          11011      Curtis          Lu                 1224 Shoenic

## Define our label

In [11]:
#create label
labels = np.array(custs['AveMonthSpend'])

## One-hot encode categorical features

In [12]:
#Create feature array
def encode_string(cat_features):
    ## First encode the strings to numeric categories
    enc = preprocessing.LabelEncoder()
    enc.fit(cat_features)
    enc_cat_features = enc.transform(cat_features)
    ## Now, apply one hot encoding
    ohe = preprocessing.OneHotEncoder()
    encoded = ohe.fit(enc_cat_features.reshape(-1,1))
    return encoded.transform(enc_cat_features.reshape(-1,1)).toarray()

categorical_columns = ['Education', 'Occupation', 'Gender', 'MaritalStatus', 'HomeOwnerFlag']

Features = encode_string(custs['CountryRegionName'])
for col in categorical_columns:
    temp = encode_string(custs[col])
    Features = np.concatenate([Features, temp], axis = 1)


print(Features.shape)
print(Features[:2, :])
print(custs['CountryRegionName'].unique())

(16471, 22)
[[1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0.]]
['Australia' 'United States' 'Canada' 'Germany' 'United Kingdom' 'France']


C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used

Let's merge the encoded categorical features with numeric features

In [13]:
#Concatenate with numeric features
Features = np.concatenate([Features, np.array(custs[
                                                  ['NumberCarsOwned', 'NumberChildrenAtHome',
                                                      'TotalChildren', 'YearlyIncome', 'Age']
                                              ])], axis = 1)
# print(Features.shape)
# print(Features[:2, :])

#Create feature array for AW_Test and concatenate numeric features
AW_test_Features = encode_string(AW_test['CountryRegionName'])
for col in categorical_columns:
    temp = encode_string(AW_test[col])
    AW_test_Features = np.concatenate([AW_test_Features, temp], axis = 1)

AW_test_Features = np.concatenate([AW_test_Features, np.array(AW_test[
                                                  ['NumberCarsOwned', 'NumberChildrenAtHome',
                                                      'TotalChildren', 'YearlyIncome', 'Age']
                                              ])], axis = 1)

print(AW_test_Features[:2,:])
print(AW_test_Features.shape)

[[0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 1.00000e+00
  1.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
  1.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 1.00000e+00 0.00000e+00
  0.00000e+00 1.00000e+00 1.00000e+00 0.00000e+00 2.00000e+00 0.00000e+00
  5.00000e+00 8.69310e+04 5.30000e+01]
 [0.00000e+00 1.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
  1.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
  0.00000e+00 0.00000e+00 0.00000e+00 1.00000e+00 0.00000e+00 1.00000e+00
  1.00000e+00 0.00000e+00 0.00000e+00 1.00000e+00 2.00000e+00 2.00000e+00
  4.00000e+00 1.00125e+05 3.30000e+01]]
(500, 27)


C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used

## Create training and test sets

In [14]:
#Split the data
nr.seed(8585)
indx = range(Features.shape[0])
indx = ms.train_test_split(indx, test_size = 500)
X_train = Features[indx[0],:]
y_train = np.ravel(labels[indx[0]])
X_test = AW_test_Features
#y_test = np.ravel(labels[indx[1]]) #no y_test

## Scale the data

In [15]:
#Scale the data
scaler = preprocessing.StandardScaler().fit(X_train[:,22:])
X_train[:,22:] = scaler.transform(X_train[:,22:])
X_test[:,22:] = scaler.transform(X_test[:,22:])
# print(X_train[:2,])

## Create the model

In [16]:
#Construct the linear regression model
lin_mod = linear_model.LinearRegression()
lin_mod.fit(X_train, y_train)
print(lin_mod.intercept_)
print(lin_mod.coef_)
y_score = lin_mod.predict(X_test)
print(y_score[:3])

-76950206890290.66
[-5.88934922e+11 -5.88934922e+11 -5.88934922e+11 -5.88934922e+11
 -5.88934922e+11 -5.88934922e+11  3.80570350e+13  3.80570350e+13
  3.80570350e+13  3.80570350e+13  3.80570350e+13  3.61794583e+13
  3.61794583e+13  3.61794583e+13  3.61794583e+13  3.61794583e+13
 -1.94736266e+12 -1.94736266e+12  1.55723440e+12  1.55723440e+12
  3.69277676e+12  3.69277676e+12 -4.20776367e-01  1.62786560e+01
  6.30022049e-01  8.61718750e+00 -1.49377441e+00]
[ 42.71875 107.       48.71875]


## Evaluate the model

In [26]:
# Setup cross validation folds
kf = KFold(n_splits=10, random_state=42, shuffle=True)

In [27]:
def cv_r2(model, X=X_train):
    r2 = cross_val_score(model, X_train, y_train, scoring="r2", cv=kf)
    return (r2)

In [28]:
scores = {}
score = cv_r2(lin_mod)
print("lin_mod: {:.4f} ({:.4f})".format(score.mean(), score.std()))
scores['lin_mod'] = (score.mean(), score.std())

lin_mod: 0.9464 (0.0021)


## Create the submission file

In [ ]:
# # Export result as csv file
# pd.DataFrame(y_score).to_csv("regression_test_result.csv")